<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

### Install Beautiful Soup package

In [56]:
!pip install bs4

### Import libraries

In [57]:
from bs4 import BeautifulSoup
import pandas as pd

### Download the wikipedia url

In [58]:
!wget https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M -O toronto_postalcodes.html

--2021-06-25 04:32:10--  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Resolving en.wikipedia.org (en.wikipedia.org)... 208.80.154.224, 2620:0:861:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|208.80.154.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 83382 (81K) [text/html]
Saving to: ‘toronto_postalcodes.html’

toronto_postalcodes 100%[===================>]  81.43K   123KB/s    in 0.7s    

2021-06-25 04:32:11 (123 KB/s) - ‘toronto_postalcodes.html’ saved [83382/83382]



### Read the wikipedia html data

In [59]:
html = ''.join(open("toronto_postalcodes.html").readlines())

### Process wikipedea data into dataframe

In [60]:
soup = BeautifulSoup(html, 'html.parser')
table_contents=[]
table = soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = row.span.text.split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

In [61]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [62]:
df.shape

(103, 3)

### Read geospatial coordinates data

In [63]:
coords = pd.read_csv('./Geospatial_Coordinates.csv')

In [64]:
coords.columns = ['PostalCode', 'Latitude', 'Longitude']
geo_df = df.merge(coords)

In [65]:
geo_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
